# Project objective
In this project, we try to identify the best k in k-nearest neighbour machine learning method for predicting tissue of origin of cancer cell lines using their gene expression provided in cancer cell line encyclopedia dataset.

This is a simple case of hyperparameter optimization. The selection is done by identifying best k comparing performance of the models in cross-validation setting.

Information about the dataset, some technical details about the used machine learning method(s) and mathematical details of the quantifications approaches are provided in the code. 

# Packages we work with in this notebook
We are going to use the following libraries and packages:

* **numpy**: NumPy is the fundamental package for scientific computing with Python. (http://www.numpy.org/)
* **sklearn**: Scikit-learn is a machine learning library for Python programming language. (https://scikit-learn.org/stable/)
* **pandas**: Pandas provides easy-to-use data structures and data analysis tools for Python. (https://pandas.pydata.org/)


In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

# Introduction to the dataset

**Name**: Cancer Cell Line Encyclopedia dataset

**Summary**: Identifying tissue of origin of cancer cell lines using their gene expression. Cell lines from 6 tissues were chosen for this code including: breast, central_nervous_system, haematopoietic_and_lymphoid_tissue, large_intestine, lung, skin

**number of features**: 500 (real, positive) 
Top 500 genex based on variance of their expression in the dataset is chosen. The right way to select the features is to do it only on the training set to eliminate information leak from test set. But to simplify the process for the sake of this teaching code, we use all the dataset.

**Number of data points (instances)**: 550

**dataset accessibility**: Dataset is available as part of PharmacoGx R package (https://www.bioconductor.org/packages/release/bioc/html/PharmacoGx.html)

**Link to the dataset**: https://portals.broadinstitute.org/ccle




## Importing the dataset
We can import the dataset in multiple ways

**Colab Notebook**: You can download the dataset file (or files) from the link (if provided) and uploading it to your google drive and then you can import the file (or files) as follows:

**Note.** When you run the following cell, it tries to connect the colab with google derive. Follow steps 1 to 5 in this link (https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/) to complete the 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# This path is common for everybody
# This is the path to your google drive
input_path = '/content/gdrive/My Drive/'
# reading the data (target)
target_dataset_features = pd.read_csv(input_path + 'CCLE_ExpMat_Top500Genes.csv', index_col=0)
target_dataset_output = pd.read_csv(input_path + 'CCLE_ExpMat_Phenotype.csv', index_col=0)
# Transposing the dataframe to put features in the dataframe columns
target_dataset_features = target_dataset_features.transpose()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Local directory**: In case you save the data in your local directory, you need to change "input_path" to the local directory you saved the file (or files) in.

**GitHub**: If you use my GitHub (or your own GitHub) repo, you need to change the "input_path" to where the file (or files) exist in the repo. For example, when I clone ***ml_in_practice*** from my GitHub, I need to change "input_path" to 'data/' as the file (or files) is saved in the data dicretory in this repository. 

**Note.**: You can also clone my ***ml_in_practice*** repository (here: https://github.com/alimadani/ml_in_practice) and follow the same process.

## Making sure about the dataset characteristics (number of data points and features)

In [ ]:
print('number of features: {}'.format(target_dataset_features.shape[1]))
print('number of data points: {}'.format(target_dataset_features.shape[0]))

number of features: 500
number of data points: 550


## Data preparation
We need to prepare the dataset for machine learnign modeling. Here we prepare the data in 2 steps:

1) Selecting target columns from the output dataframe (target_dataset_output)
2) Converting tissue names to integers (one for each tissue)

In [ ]:
# tissueid is the column that contains tissue type information
output_var_names = target_dataset_output['tissueid']
# converting tissue names to labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(output_var_names)
output_var = le.transform(output_var_names)

# we would like to use all the features as input features of the model
input_features = target_dataset_features

## Splitting data to training and testing sets

We need to split the data to train and test, if we do not have a separate dataset for validation and/or testing, to make sure about generalizability of the model we train.

**test_size**: Traditionally, 30%-40% of the dataset cna be used for test set. If you split the data to train, validation and test, you can use 60%, 20% and 20% of the dataset, respectively.

**Note.**: We need the validation and test sets to be big enough for checking generalizability of our model. At the same time we would like to have as much data as possible in the training set to train a better model.

**random_state** as the name suggests, is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices in your case.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_features, output_var, test_size=0.30, random_state=5)

## Building the supervised learning model
We want to build a multi-class classification model as the output variable include multiple classes. Here we build three models using naive Bayes, k nearest neighbour and random forest algorithms.


### k-nearest neighbours(kNN)
k-nearest neighbours uses a distance metric like Euclidean distance to identity similarity of a data point (sample) to the other data points (samples) in the trainign set. Then based on the user specified k, it finds the k closest points (samples) to the target data point. Afterward, it chooses the most frequent label among the k closes points (majority voting) as the class label of the target sample. The class labels can be also assigned based on weighted voting of the k closest data points to the data point. 

This process is basis of identifying regions in the space belong to each class. For small k (k=1 or 2) the space may look like collection of islands belonging to diffeerent classes. While getting to higher k values make the islands connected to become class territories.


## Cross-validation and checking generalizability of the model
After training a machine learning model, we need to check its generalizability and making sure it is not just good in prediction of training set but is capable of predicting new data points. In previous projects we splitted the data to 2 parts, training and test set. We can go one step further and repeat this splitting across the dataset so that every single data point is considered in one of the test (better to be said validation) sets. This process is called k-fold cross-validation. For example in case of 5-fold cross-validation, the dataset is splitted to 5 chunks and the model is trained in 4 out of 5 chunk and tested on the remianing chunk. The test chunk is then rotated so taht every chunk is conisidered once for testing the model. Then we can get average performance of the model in the tested chunks.

Here we use 5-fold cross-validation.

Note. Lack (or low level) of generalizability of a trained model to new data points is called overfitting.

## Hyperparameter selection
We have parameters and hyperparameters that need to be determined to build a machine learning model. The parameters are determined in the optimization process in training set (this is hat happens when we train a model). The hyperparameters are those exist for the method (like k in kNN) irrespect of the data. But these hyperparameters can be optimized for the dataset at hand. The hyperparameter optimization is usually done in validation (or development) set. In cross-validation, we are technically assesing performanc of a model at hand on different validation sets we have in cross-validation setting. Hence, the performance in cross-validation setting can be compared to select the best hypeparameters.

**Note.** There is a grid search function in sklearn for hypeparameter selection that we will review in future projects.


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

k_hyperparam = [3, 5, 10, 15]
scores = []
for k_iter in k_hyperparam:
  knn = KNeighborsClassifier(n_neighbors=k_iter, weights='distance')
  scores.append(round(cross_val_score(knn, X_train, y_train, cv=5).mean(), 3))
  
# average performance across all folds
print("Average Accuracy in 5-fold cross validation for k values of [3, 5, 10, 15] are {}, respectively.".format(scores))


Average Accuracy in 5-fold cross validation for k values of [3, 5, 10, 15] are [0.873, 0.882, 0.871, 0.869], respectively.


We identified that k=5 results in the best performance (average accuracy=0.882) in 5-fold cross-validation setting. Now we use all teh training data to refit a kNN model with k=5 and then assess the performance of the model in the test set.

**Note.** Although we chose the hyperparameters for the best model, the differences between the performance of the models is not significant. We will quantify significance of the difference between performance of the models in future projects.

In [ ]:
# Create k nearest neighbour object
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
# Train the models using the training sets
knn.fit(X_train, y_train)
# Make predictions using the testing set
y_pred_knn = knn.predict(X_test)

## Evaluating performance of the model
We need to assess performance of the model using the predictions of the test set. We use accuracy and balanced accuracy. Here are their definitions:

* **recall** in this context is also referred to as the true positive rate or sensitivity

How many relevant item are selected




$${\displaystyle {\text{recall}}={\frac {tp}{tp+fn}}\,} $$

 

* **specificity** true negative rate



$${\displaystyle {\text{true negative rate}}={\frac {tn}{tn+fp}}\,}$$

* **accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$$ {\displaystyle {\text{accuracy}}={\frac {tp+tn}{tp+tn+fp+fn}}\,}$$


\begin{equation*} accuracy=\frac{number\:of\:correct\:predictions}{(total\:number\:of\:data\:points (samples))} \end{equation*}


* **balanced accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$${\displaystyle {\text{balanced accuracy}}={\frac {recall+specificity
}{2}}\,}$$


In [ ]:
from sklearn import metrics

print("accuracy of the predictions using kNN with k=5:", metrics.accuracy_score(y_test, y_pred_knn))

print("blanced accuracy of the predictions using kNN with k=5:", metrics.balanced_accuracy_score(y_test, y_pred_knn))

accuracy of the predictions using kNN with k=5: 0.9272727272727272
blanced accuracy of the predictions using kNN with k=5: 0.889917852615221
